# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load and clean weather data
weather_data_to_load = os.path.join("resources", "City_Data.csv")
weather_data = pd.read_csv(weather_data_to_load)

index_names = weather_data[weather_data['Humidity'] > 100].index
clean_weather_data = weather_data.drop(index_names, inplace=False)

clean_weather_data.dropna(inplace=True)

In [3]:
# Load the csv to a DataFrame
weather_data_to_load = os.path.join("resources", "City_Data.csv")
weather_data = pd.read_csv(weather_data_to_load)

#Cleaning weather data by removing data with humidity more than 100
index_names = weather_data[ weather_data['Humidity'] > 100 ].index
clean_weather_data = weather_data.drop(index_names, inplace = False)

#drop na values
clean_weather_data.dropna(inplace = True) 

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = clean_weather_data[["Lat", "Long"]]

# Fill NaN values and convert to float
humidity = clean_weather_data["Humidity"].astype(float)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(20.0, 10.0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2.5)
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Filtering data with temperature > 21 and < 27
index_names = clean_weather_data[ (clean_weather_data['Temp'] > 21) & (clean_weather_data['Temp'] < 27)].index
fit_weather_data = clean_weather_data.drop(index_names, inplace = False)

#Filtering data with wind speed less than 4.5 m/s
index_names = fit_weather_data[ fit_weather_data['Wind'] <= 4.5].index
fit_weather_data = fit_weather_data.drop(index_names, inplace = False)

#Filtering data with zero cloudiness
index_names = fit_weather_data[ fit_weather_data['Clouds'] > 0].index
fit_weather_data = fit_weather_data.drop(index_names, inplace = False)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = fit_weather_data[['City', 'Country', 'Lat', 'Long', 'Humidity']]
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 10000,
    "keyword": "hotel",
    "type": "hotel",
    "key": g_key
    }
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get city from df
    city = row['City']
    params['location'] = f'{row["Lat"]},{row["Long"]}'

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("na")
hotel_df

/Users/liamtwomey/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/liamtwomey/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Long,Humidity,Hotel Name
27,lebu,CL,-37.6167,-73.6500,63,Hotel Plaza Lebu
38,general roca,AR,-39.0333,-67.5833,18,Hotel Alamo Inn
43,hamilton,US,39.1834,-84.5333,56,Hilton Cincinnati Netherland Plaza
48,arraial do cabo,BR,-22.9661,-42.0278,70,Orlanova Boutique Hotel
112,fairview,US,40.8126,-73.9990,46,West Farms Hotel
121,pascagoula,US,30.3658,-88.5561,45,Hilton Garden Inn Pascagoula
152,olga,RU,43.7444,135.2892,67,Gostinitsa Rosmorport.
153,lewistown,US,40.5992,-77.5714,70,Quality Inn & Suites
173,shitanjing,CN,39.2342,106.3439,31,Xin Xi Lai Shun Hotel
175,gornyak,RU,50.9942,81.4661,71,"Gostinitsa ""Yubileynaya"""


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))